In [0]:
from fastai import *
from fastai.tabular import *
import pandas as pd
from torchsummary import summary
import torch
from torch import nn
import imageio
import torch
import glob
from fastai.vision import *
import os
from torch import nn
import torch.nn.functional as F
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)
%cd "/content/drive/My Drive/automatic-asset-classification"
%ls "/content/drive/My Drive/automatic-asset-classification"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/automatic-asset-classification
data/


In [0]:
np.random.seed(3333)
torch.manual_seed(3333)
image_path = "/content/drive/My Drive/automatic-asset-classification/data/final_dataset"
size = 224
batchsize = 32

tfms = get_transforms(do_flip = True)
src = (ImageImageList.from_folder(image_path).split_by_rand_pct().label_from_func(lambda x: x))
data = (src.transform(tfms, size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats, do_y = False))

In [ ]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor = 2, mode = 'bilinear'),
            nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.ReLU(inplace=True)
        )
        
    def encode(self,x): return self.encoder(x)
    
    def decode(self,x): return torch.clamp(self.decoder(x), min = 0, max=1)
        

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return torch.clamp(decoded, min=0, max=1)

In [0]:
autoencoder = AutoEncoder()
learn = Learner(data, autoencoder, loss_func=F.mse_loss);
learn = learn.load('ae_12_31_3_2020')

In [0]:
train_batches = [item[0] for item in iter(data.train_dl)]
valid_batches = [item[0] for item in iter(data.valid_dl)]

In [0]:
autoencoder.eval()
encoded_train_batches = [];
encoded_valid_batches = [];

with torch.no_grad():

  for train in train_batches:
    encoded_train = autoencoder.encode(train);
    encoded_train_batches.append(encoded_train);
  
  for valid in valid_batches:
    encoded_valid = autoencoder.encode(valid);
    encoded_valid_batches.append(encoded_valid);

In [0]:
del train_batches
del valid_batches
del data
del src
del autoencoder
del learn

In [0]:
#encoded_train_batches = torch.cat([batch.cpu() for batch in encoded_train_batches])
#encoded_valid_batches = torch.cat([batch.cpu() for batch in encoded_valid_batches])

encoded_train_batches = np.concatenate([batch.cpu().numpy() for batch in encoded_train_batches])
encoded_valid_batches = np.concatenate([batch.cpu().numpy() for batch in encoded_valid_batches])

In [0]:
dats = ItemList(encoded_train_batches).split_by_rand_pct()

In [0]:
dats.label_from_lists(dats.train, dats.valid)

In [0]:
class encodedFloatItem(ItemBase):
    "Basic class for float items."
    def __init__(self,obj): self.data,self.obj = np.array(obj).astype(np.float32),obj
    def __str__(self):  return str(self.data.shape)
    def __hash__(self): return hash(str(self))


In [57]:
encodedFloatItem(encoded_train_batches[0])

encodedFloatItem (64, 112, 112)

In [0]:
class encodedList(FloatList):
    def __init__(self, items:Iterator, log:bool=False, **kwargs):
        if isinstance(items, ItemList):
            items = items.items
        super(FloatList,self).__init__(items,**kwargs)
    def get(self,i):
        return encodedFloatItem(torch.Tensor(super(FloatList,self).get(i).astype('float32')))

In [0]:
ae_dat = encodedList(encoded_train_batches).split_none()

In [66]:
ae_dat

ERROR! Session/line number was not unique in database. History logging moved to new session 62


ItemLists;

Train: encodedList (384 items)
(64, 112, 112),(64, 112, 112),(64, 112, 112),(64, 112, 112),(64, 112, 112)
Path: .;

Valid: encodedList (0 items)

Path: .;

Test: None

In [0]:
train_dat = encodedList(encoded_train_batches)

In [0]:
ae_dat.valid = encodedList(encoded_valid_batches)

In [0]:
ae_dat.label_from_lists(train_dat, encodedList(encoded_valid_batches))

In [0]:
ae_dat = ItemList(encoded_train_batches).split_none()
ae_dat.valid = encoded_valid_batches

In [0]:
del encoded_train_batches
del encoded_valid_batches

In [0]:
ae_dat.label_from_lists(ae_dat.train, ae_dat.valid)

In [0]:
class encodedList(FloatList):
    def __init__(self, items:Iterator, log:bool=False, **kwargs):
        if isinstance(items, ItemList):
            items = items.items
        super(FloatList,self).__init__(items,**kwargs)
    def get(self,i):
        return torch.Tensor(super(FloatList,self).get(i).astype('float32'))

In [22]:
ae_dat = encodedList(encoded_train_batches).split_none()
ae_dat.valid = encodedList(encoded_valid_batches)

NameError: ignored

In [21]:
ae_dat.

RuntimeError: ignored

In [0]:
ae_dat.label_from_func(lambda x: x)